In [14]:
import requests

url = "https://behnam-torob-ai-shopping-assistant.darkube.app/chat"
data = {
    "chat_id": "1",
    "messages": [{"type": "text", "content": "return base random key: 5"}]
}

# Disable system proxy
resp = requests.post(url, json=data, proxies={"http": None, "https": None})
print(resp.json())

{'message': None, 'base_random_keys': ['5'], 'member_random_keys': None}
